Notebook created on 16 October 2021 by Chris Mulvey  
For NSS Data Science Bootcamp

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup as BS
from tqdm.notebook import tqdm
import time

### Working out a function to get all of the districts in a state

#### Reading in state and district info from a previous webscrape

In [2]:
districts = pd.read_csv('../data/district_info.csv')

districts.head()

,state,representatives,Code
0,Alabama,7,AL
1,Alaska,1,AK
2,Arizona,9,AZ
3,Arkansas,4,AR
4,California,53,CA


### The URL has state and district number in it `id=TN02`

In [ ]:
url = 'https://www.opensecrets.org/races/summary?cycle=2020&id=TN02&spec=N'

In [ ]:
url = 'https://www.opensecrets.org/races/summary?cycle=2020&id='+ state + district'&spec=N'

### Function to get candidate info for a single state and district

In [13]:
def get_district_candidates(year, state, district):
    url = 'https://www.opensecrets.org/races/summary?cycle='+ year + '&id='+ state + district + '&spec=N'
    response = requests.get(url)
    soup = BS(response.text)
    try:
        candidates = pd.read_html(str(soup.find('table')))[0]
    except:
        print(response.headers) #prints out the header to help troubleshoot HTTP errors
    candidates['state'] = state
    candidates['district'] = district
    candidates['year'] = year
    
    candidates = fix_columns(candidates)
    
    return candidates

In [14]:
get_district_candidates('2016', 'TN', str('0' + '3'))

,candidate,party,state,district,incumbent,winner,raised,spent,year
0,Chuck Fleischmann,R,TN,03,Incumbent,Winner,"$1,421,141","$924,617",2016
1,Melody Shekari,D,TN,03,NaN,NaN,"$97,219","$97,219",2016


### Now to bring in more states

In [ ]:
state_dist = pd.DataFrame()

for index in tqdm(districts.index):
    state = districts['Code'][index]
    dist_no = districts['representatives'][index]
    
    if index == 10:
        time.sleep(70)
    elif index == 20:
        time.sleep(70)
    elif index == 30:
        time.sleep(70)
    elif index == 40:
        time.sleep(70)
    
    for i in range(1, dist_no + 1):
        if i < 10:
            i = '0' + str(i)
            
        state_dist = state_dist.append(get_district_candidates(str(state), str(i)),
                                      ignore_index = True)
            

In [ ]:
state_dist.info()

In [ ]:
state_dist.to_csv('../data/all_state.csv',
                 index = False)

### Troubleshooting error

In [ ]:
districts.loc[43]

In [ ]:
for i in range(1, 4):
    if i < 10:
        i = '0' + str(i)
        
    print(get_district_candidates('UT', str(i)))

In [ ]:
i = 3

if i < 10:
    i = '0' + str(i)
    print(str(i))

## Building function to properly set up columns

In [12]:
def fix_columns(df):
    df['party'] = df['Candidate'].str.extract(r'\((\w)\)')
    df['incumbent'] = df['Candidate'].str.extract(r'([I]\w+)')
    df['winner'] = df['Candidate'].str.extract(r'([W]\w+)')
    df['candidate'] = df['Candidate'].str.extract(r'(^\w+\s\w+\s*\w+)')

    df = df.drop('Candidate',
                 axis = 1)
    
    df = df[['candidate',
             'party',
             'state',
             'district',
             'incumbent',
             'winner',
             'Raised',
             'Spent',
             'year']]
         
    df = df.rename(columns = {
        'Raised': 'raised',
        'Spent': 'spent'
    })
    
    return df

## Building a function to run it all and get data for other years as well

In [35]:
def get_all_reps_time(df, st_col, rep_col, year):
    """Takes in a DataFrame that holds the number of representatives per state and the year to
    pull data for from opersecrets.org and returns a dataframe of nationwide candidates and
    associated info.
    
    df: Variable name for the DataFrame that holds the state and number of Reps
    st_col: the DataFrame column that holds the two-digit state code
    rep_col: the DataFrame column that holds the number of reps
    year: Needs to be a string and 4 digits.
    """
    
    # Defining the function that will set up the columns in the final dataframe
    def fix_columns(df):
        df['party'] = df['Candidate'].str.extract(r'\((\w)\)')
        df['incumbent'] = df['Candidate'].str.extract(r'(Incumbent)')
        df['winner'] = df['Candidate'].str.extract(r'(Winner)')
        df['candidate'] = df['Candidate'].str.extract(r'(^\w+\s\w+\s*\w+)')

        df = df.drop('Candidate',
                 axis = 1)
    
        df = df[['candidate',
                 'party',
                 'state',
                 'district',
                 'incumbent',
                 'winner',
                 'Raised',
                 'Spent',
                 'year']]
         
        df = df.rename(columns = {
            'Raised': 'raised',
            'Spent': 'spent'
        })
        
        df['raised'] = df['raised'].str.replace(r'\$|,', '', regex=True).astype('int64')
        df['spent'] = df['spent'].str.replace(r'\$|,', '', regex=True).astype('int64')
    
        return df
    
    # Defining the function that will webscrape the candidate info
    def get_district_candidates(year, state, district):
        url = 'https://www.opensecrets.org/races/summary?cycle='+ year + '&id='+ state + district + '&spec=N'
        response = requests.get(url)
        soup = BS(response.text)
        try:
            candidates = pd.read_html(str(soup.find('table')))[0]
        except:
            print(response.headers) #prints out the header to help troubleshoot HTTP errors
        candidates['state'] = state
        candidates['district'] = district
        candidates['year'] = year
    
        candidates = fix_columns(candidates)
    
        return candidates
    
    # Setting up the for loop to get all states for a defined year
    
    state_dist = pd.DataFrame() # Empty DataFrame for the for loop

    for index in tqdm(df.index):
        state = df[st_col][index]
        dist_no = df[rep_col][index]
    
        if index == 10:
            time.sleep(70)
        elif index == 20:
            time.sleep(70)
        elif index == 30:
            time.sleep(70)
        elif index == 40:
            time.sleep(70)
    
        for i in range(1, dist_no + 1):
            if i < 10:
                i = '0' + str(i)
            
            state_dist = state_dist.append(get_district_candidates(str(year), str(state), str(i)),
                                          ignore_index = True)
    return state_dist

### Testing the function

In [36]:
reps_2016 = get_all_reps_time(districts, 'Code', 'representatives', '2016')

  0%|          | 0/50 [00:00<?, ?it/s]

In [37]:
reps_2016

,candidate,party,state,district,incumbent,winner,raised,spent,year
0,Bradley Byrne,R,AL,01,Incumbent,Winner,1366370,1171650,2016
1,Martha Roby,R,AL,02,Incumbent,Winner,1381821,1828096,2016
2,Mike D Rogers,R,AL,03,Incumbent,Winner,1137285,1069552,2016
3,Jesse Smith,D,AL,03,NaN,NaN,9810,7348,2016
4,Robert B Aderholt,R,AL,04,Incumbent,Winner,1433641,1387066,2016
...,...,...,...,...,...,...,...,...,...
786,Mary Hoeft,D,WI,07,NaN,NaN,116276,116276,2016
787,Mike Gallagher,R,WI,08,NaN,Winner,2757473,2695648,2016
788,Tom Nelson,D,WI,08,NaN,NaN,1804001,1751957,2016
789,Liz Cheney,R,WY,01,NaN,Winner,2206006,2148256,2016


In [39]:
reps_2016['winner'].value_counts()

Winner    435
Name: winner, dtype: int64

In [40]:
reps_2016.to_csv('../data/reps_2016.csv',
                index = False)

### Pulling every two years between 2010 and 2020

In [41]:
reps_2010 = get_all_reps_time(districts, 'Code', 'representatives', '2010')

  0%|          | 0/50 [00:00<?, ?it/s]

In [42]:
reps_2010

,candidate,party,state,district,incumbent,winner,raised,spent,year
0,Jo Bonner,R,AL,01,Incumbent,Winner,913053,1101701,2010
1,David Walter,3,AL,01,NaN,NaN,21321,20045,2010
2,Bobby Bright,D,AL,02,Incumbent,NaN,1413032,1435526,2010
3,Martha Roby,R,AL,02,NaN,Winner,1253557,1240276,2010
4,Mike D Rogers,R,AL,03,Incumbent,Winner,1141732,943060,2010
...,...,...,...,...,...,...,...,...,...
942,Julie Lassa,D,WI,07,NaN,NaN,1271594,1270399,2010
943,Steve Kagen,D,WI,08,Incumbent,NaN,2076971,2080786,2010
944,Reid Ribble,R,WI,08,NaN,Winner,1295833,1287780,2010
945,Cynthia Lummis,R,WY,01,Incumbent,Winner,780426,576836,2010


In [43]:
reps_2010['winner'].value_counts()

Winner    424
Name: winner, dtype: int64

In [44]:
reps_2010.to_csv('../data/resp_2010.csv',
                index = False)

In [45]:
reps_2012 = get_all_reps_time(districts, 'Code', 'representatives', '2012')

  0%|          | 0/50 [00:00<?, ?it/s]

In [46]:
reps_2012

,candidate,party,state,district,incumbent,winner,raised,spent,year
0,Jo Bonner,R,AL,01,Incumbent,Winner,1140567,1263168,2012
1,Curtis Railey,I,AL,01,NaN,NaN,18015,15646,2012
2,Martha Roby,R,AL,02,Incumbent,Winner,1035717,862502,2012
3,Mike D Rogers,R,AL,03,Incumbent,Winner,1069891,982102,2012
4,John Andrew Harris,D,AL,03,NaN,NaN,3147,3786,2012
...,...,...,...,...,...,...,...,...,...
865,Dale Lehner,I,WI,07,NaN,NaN,1118,842,2012
866,Reid Ribble,R,WI,08,Incumbent,Winner,2320569,2086968,2012
867,Jamie Wall,D,WI,08,NaN,NaN,1066991,1048910,2012
868,Cynthia Lummis,R,WY,01,Incumbent,Winner,715314,593403,2012


In [47]:
reps_2012.to_csv('../data/reps_2012.csv',
                index = False)

In [48]:
reps_2014 = get_all_reps_time(districts, 'Code', 'representatives', '2014')

reps_2014

  0%|          | 0/50 [00:00<?, ?it/s]

,candidate,party,state,district,incumbent,winner,raised,spent,year
0,Bradley Byrne,R,AL,01,Incumbent,Winner,1907597,1655999,2014
1,Burton Leflore,D,AL,01,NaN,NaN,36974,35603,2014
2,Martha Roby,R,AL,02,Incumbent,Winner,1098795,793565,2014
3,Erick Wright,D,AL,02,NaN,NaN,6674,10473,2014
4,Mike D Rogers,R,AL,03,Incumbent,Winner,1102097,936332,2014
...,...,...,...,...,...,...,...,...,...
820,Kelly Westlund,D,WI,07,NaN,NaN,528203,526935,2014
821,Reid Ribble,R,WI,08,Incumbent,Winner,1703029,1066298,2014
822,Ron Gruett,D,WI,08,NaN,NaN,23113,22206,2014
823,Cynthia Lummis,R,WY,01,Incumbent,Winner,432666,300949,2014


In [49]:
reps_2014.to_csv('../data/reps_2014.csv',
                index = False)

In [50]:
reps_2018 = get_all_reps_time(districts, 'Code', 'representatives', '2018')

reps_2018

  0%|          | 0/50 [00:00<?, ?it/s]

,candidate,party,state,district,incumbent,winner,raised,spent,year
0,Bradley Byrne,R,AL,01,Incumbent,Winner,1460041,831634,2018
1,Robert Kennedy Jr,D,AL,01,NaN,NaN,46845,46845,2018
2,Martha Roby,R,AL,02,Incumbent,Winner,2559441,2263209,2018
3,Tabitha Isner,D,AL,02,NaN,NaN,519145,519145,2018
4,Mike D Rogers,R,AL,03,Incumbent,Winner,1349986,1347847,2018
...,...,...,...,...,...,...,...,...,...
860,Margaret Engebretson,D,WI,07,NaN,NaN,120738,118724,2018
861,Mike Gallagher,R,WI,08,Incumbent,Winner,2817157,2070725,2018
862,Beau Liegeois,D,WI,08,NaN,NaN,356744,342250,2018
863,Liz Cheney,R,WY,01,Incumbent,Winner,867571,715470,2018


In [51]:
reps_2018.to_csv('../data/reps_2018.csv',
                index = False)

In [52]:
reps_2020 = get_all_reps_time(districts, 'Code', 'representatives', '2020')

reps_2020

  0%|          | 0/50 [00:00<?, ?it/s]

,candidate,party,state,district,incumbent,winner,raised,spent,year
0,Jerry Carl,R,AL,01,NaN,Winner,1971321,1859349,2020
1,James Averhart,D,AL,01,NaN,NaN,80095,78973,2020
2,Barry Moore,R,AL,02,NaN,Winner,650807,669368,2020
3,Phyllis Harvey,D,AL,02,NaN,NaN,56050,55988,2020
4,Mike D Rogers,R,AL,03,Incumbent,Winner,1193111,1218564,2020
...,...,...,...,...,...,...,...,...,...
889,Tricia Zunker,D,WI,07,NaN,NaN,1261957,1232690,2020
890,Mike Gallagher,R,WI,08,Incumbent,Winner,3202905,2841801,2020
891,Amanda Stuck,D,WI,08,NaN,NaN,416978,399916,2020
892,Liz Cheney,R,WY,01,Incumbent,Winner,3003883,3060167,2020


In [53]:
reps_2020.to_csv('../data/reps_2020.csv',
                index = False)